# Poor man's agent

## Setup

In [ ]:
import json
import os
from collections.abc import Callable

from claudette import Client, mk_msgs, models
from openrouter import OpenRouter

from poor_mans_agent.config import get_config
from poor_mans_agent.tools import read_url, search

In [ ]:
config = get_config()

In [ ]:
config.model_dump()

## Agent

In [ ]:
prompt = "What are the top 3 news on the German Tagesschau web site for today?"

### Based on Claudette

In [ ]:
os.environ["ANTHROPIC_API_KEY"] = config.anthropic_key.get_secret_value()

In [ ]:
models

In [ ]:
client = Client(model="claude-haiku-4-5")

In [ ]:
tools = [read_url]

In [ ]:
response = client(prompt, tools=tools)

In [ ]:
response

In [ ]:
response.model_dump()

In [ ]:
tool_use_block = response.content[1]
tool_use_block

In [ ]:
tool_use_block.name

In [ ]:
def _get_tool(name: str):
    return globals().get(name)

In [ ]:
_tool = _get_tool(tool_use_block.name)
_tool

In [ ]:
def _call_tool(tool, input):
    return tool(**input)

In [ ]:
result = _call_tool(tool=_tool, input=tool_use_block.input)

In [ ]:
result.text

In [ ]:
def _format_result(id: str, result: bytes):
    return {"type": "tool_result", "tool_use_id": id, "content": json.dumps(result.text)}

In [ ]:
tool_call_result = _format_result(id=tool_use_block.id, result=result)
tool_call_result

In [ ]:
messages = mk_msgs([prompt, response.content, [tool_call_result]])
messages

In [ ]:
response = client(messages)

In [ ]:
response

#### Putting it all together

In [ ]:
def exec_agent(client: Client, prompt: str):
    response = client(prompt, tools=[search, read_url])
    while response.stop_reason == "tool_use":
        tool_use_block = response.content[1]
        _tool = _get_tool(tool_use_block.name)
        print(f"Calling tool: {tool_use_block.name}")
        print(f"Passing params: {tool_use_block.input}")
        result = _call_tool(tool=_tool, input=tool_use_block.input)
        tool_call_result = _format_result(id=tool_use_block.id, result=result)
        print(f"Tool call result is: {tool_call_result}")
        messages = mk_msgs([prompt, response.content, [tool_call_result]])
        response = client(messages)

    return response

In [ ]:
response = exec_agent(
    client=client, prompt="What is the byteCare company from Aachen in Germany doing?"
)
response

In [ ]:
response = exec_agent(
    client=client,
    prompt="Who is the founder of byteCare UG from Aachen and what has he been doing so far?",
)
response

### Based on OpenRouter

In [ ]:
client = OpenRouter(api_key=config.ai_model_key.get_secret_value())

In [ ]:
def _get_json_type(param_type):
    _mapping = {str: "string", int: "integer", float: "number", bool: "boolean"}

    _type = _mapping.get(param_type, "string")

    return _type

In [ ]:
def get_tool_schema(tool: Callable) -> dict:
    params = {}

    for k, v in tool.__annotations__.items():
        if k == "return":
            continue
        params[k] = {"type": _get_json_type(v)}

    return {
        "type": "function",
        "function": {
            "name": tool.__name__,
            "description": tool.__doc__,
            "parameters": {
                "type": "object",
                "properties": params,
                "required": [k for k in params.keys()],
            },
        },
    }

In [ ]:
tools = [get_tool_schema(t) for t in [search, read_url]]
tools

In [ ]:
response = client.chat.send(
    model=config.ai_model_name,
    tools=tools,
    messages=[{"role": "user", "content": prompt}],
)

In [ ]:
response.model_dump()

In [ ]:
response.choices[0].finish_reason

In [ ]:
response.choices[0].message.tool_calls[0]

In [ ]:
tool_use_block = response.choices[0].message.tool_calls[0]
tool_use_block

In [ ]:
tool_use_block.function.name

In [ ]:
tool_use_block.function.arguments

In [ ]:
tool_use_block.id

In [ ]:
def _format_result(id: str, result):
    return {"role": "tool", "tool_call_id": id, "content": result.text}

In [ ]:
response.choices[0].message.model_dump()

In [ ]:
def exec_or_agent(client: OpenRouter, prompt: str):
    _messages = [{"role": "user", "content": prompt}]
    response = client.chat.send(model=config.ai_model_name, tools=tools, messages=_messages)
    while response.choices[0].finish_reason == "tool_calls":
        tool_use_block = response.choices[0].message.tool_calls[0]
        _tool_name = tool_use_block.function.name
        _tool_params = json.loads(tool_use_block.function.arguments)

        _tool = _get_tool(_tool_name)
        print(f"Calling tool: {_tool_name}")
        print(f"Passing params: {_tool_params}")

        result = _call_tool(tool=_tool, input=_tool_params)
        print(result)
        tool_call_result = _format_result(id=tool_use_block.id, result=result)
        print(f"Tool call result is: {tool_call_result}")

        _messages.append(tool_call_result)

        response = client.chat.send(model=config.ai_model_name, tools=tools, messages=_messages)

    return response

In [ ]:
exec_or_agent(client, prompt=prompt)